## To-Do

* Entrenar con menos variables, ver la importancia de las variables en RL y RF.
* Entrenar por municipios y/o barrios.

In [1]:
# imports

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import numpy as np
import pandas
from sklearn.metrics import mean_absolute_error

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
folder = '/content/drive/MyDrive/MaestriaDataScience/Tesis/final_dataset_2025-09-08.xlsx'

In [4]:
import pandas as pd

data = pd.read_excel(folder)

In [5]:
data.head()

,url,precio,Baños,Superficie,Superficie Construida,Superficie Terreno,Garage,Dormitorio,barrio,municipio,precio_por_m2_construido,precio_por_m2_terreno,precio_por_m2
0,https://www.casasymas.com.uy/propiedad/183266-...,850000,3,429.0,256.0,429.0,4,4,carrasco,E,3320.312500,1981.351981,1981.351981
1,https://www.casasymas.com.uy/propiedad/940-cas...,380000,3,130.0,130.0,150.0,1,3,carrasco,E,2923.076923,2533.333333,2923.076923
2,https://www.casasymas.com.uy/propiedad/159162-...,119000,1,380.0,62.0,380.0,0,2,tres-cruces,B,1919.354839,313.157895,313.157895
3,https://www.casasymas.com.uy/propiedad/133140-...,1780000,4,263.0,263.0,1316.0,3,3,san-nicolas,E,6768.060837,1352.583587,6768.060837
4,https://www.casasymas.com.uy/propiedad/205399-...,370000,3,497.0,195.0,497.0,1,3,buceo,CH,1897.435897,744.466801,744.466801


## Funciones Auxiliares

Para automatizar entrenamiento de modelos y ahorrar tiempo. Luego invocaremos las funciones para entrenar.

In [6]:
def split_data_log(df, features: list, target: list):
  X = features
  y = np.log(target)
  X = X.fillna(0)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  return X_train, X_test, y_train, y_test

In [69]:
def split_data_log_one_hot(df, target: str, variables_numericas: list, variables_categoricas: list = None,
                           test_size: float = 0.2, random_state: int = 42):
    # Target en log
    y = np.log(df[target])

    # Si hay categóricas
    if variables_categoricas and len(variables_categoricas) > 0:
        X = df[variables_numericas + variables_categoricas].fillna(0)

        encoder = OneHotEncoder(sparse_output=False, drop="first", handle_unknown="ignore")
        encoded = encoder.fit_transform(X[variables_categoricas])
        encoded_df = pd.DataFrame(
            encoded,
            columns=encoder.get_feature_names_out(variables_categoricas),
            index=X.index
        )

        X_encoded = pd.concat([X[variables_numericas], encoded_df], axis=1)

    else:
        X_encoded = df[variables_numericas].fillna(0)
        encoder = None

    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(
        X_encoded, y, test_size=test_size, random_state=random_state
    )

    # Guardar índices originales
    test_indices = X_test.index

    return X_train, X_test, y_train, y_test, encoder, test_indices

In [8]:
def linear_regression_model(X_train, y_train):
  model_lr = LinearRegression()
  model_lr.fit(X_train, y_train)
  return model_lr

In [9]:
def lr_grid_search(X_train, y_train):

    # Definir el modelo base
    lr = LinearRegression()

    # Grilla de hiperparámetros
    param_grid = {
        'fit_intercept': [True, False],
        # 'normalize': [True, False]  # solo si usás sklearn < 1.0
    }

    # Configurar GridSearch
    grid_search = GridSearchCV(
        estimator=lr,
        param_grid=param_grid,
        cv=5,
        scoring='neg_mean_absolute_error',
        n_jobs=-1
    )

    # Entrenar
    grid_search.fit(X_train, y_train)

    # Retornar el mejor modelo
    print("Mejores parámetros:", grid_search.best_params_)
    print("Mejor score (CV):", grid_search.best_score_)

    return grid_search.best_estimator_


In [10]:
def evaluate_model(model, X_test, y_test):
    y_test_pred_log = model.predict(X_test)

    y_test_pred_real = np.exp(y_test_pred_log)
    y_test_real = np.exp(y_test)

    mse_real = mean_squared_error(y_test_real, y_test_pred_real)
    r2_real = r2_score(y_test_real, y_test_pred_real)
    mae_real = mean_absolute_error(y_test_real, y_test_pred_real)
    medae_real = np.median(np.abs(y_test_real - y_test_pred_real))

    # MAPE — Mean Absolute Percentage Error
    mape = 100 * np.mean(np.abs((y_test_real - y_test_pred_real) / y_test_real))

    # MdAPE — Median Absolute Percentage Error
    mdape = 100 * np.median(np.abs((y_test_real - y_test_pred_real) / y_test_real))

    return mse_real, r2_real, mae_real, medae_real, mape, mdape

In [11]:
def train_random_forest(X_train, y_train):
    rf = RandomForestRegressor(random_state=42, n_jobs=-1)
    rf.fit(X_train, y_train)
    return rf

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

def random_forest_grid_search(X_train, y_train, cv: int = 5, n_jobs: int = -1, verbose: int = 2):

    rf = RandomForestRegressor(random_state=42, n_jobs=-1)

    param_grid = {
        'n_estimators': [200, 400, 600],        # número de árboles
        'max_depth': [None, 10, 20],            # profundidad máxima
        'min_samples_split': [2, 10, 20],       # min de muestras para dividir nodo
        'min_samples_leaf': [1, 5, 10],         # min de muestras por hoja
        'max_features': ['sqrt', 'log2', 0.8]   # nº de features consideradas por split
    }

    grid_search = GridSearchCV(
        estimator=rf,
        param_grid=param_grid,
        cv=cv,
        n_jobs=n_jobs,
        scoring='neg_mean_absolute_error',
        verbose=verbose
    )

    grid_search.fit(X_train, y_train)

    print("Mejores parámetros:", grid_search.best_params_)
    print("Mejor score (CV MAE log):", grid_search.best_score_)

    return grid_search.best_estimator_

In [119]:
def evaluation_table(model, X_test, y_test, original_df, extra_cols: list = None):

    # Predicciones
    y_pred_log = model.predict(X_test)
    y_pred_real = np.exp(y_pred_log)
    y_test_real = np.exp(y_test)

    # Crear DataFrame con errores
    results = pd.DataFrame({
        "Precio Real": y_test_real,
        "Precio Predicho": y_pred_real,
        "Error Absoluto": np.abs(y_test_real - y_pred_real),
        "Error %": 100 * np.abs((y_test_real - y_pred_real) / y_test_real)
    }, index=y_test.index)

    # Agregar columnas adicionales si existen en original_df
    if extra_cols:
        original_df = original_df.reset_index(drop=True)
        for col in extra_cols:
            if col in original_df.columns:
                results[col] = original_df[col].values[:len(results)]

    results = results.sort_values(by="Error %", ascending=False)

    return results.reset_index(drop=True)

In [ ]:
def evaluate_model(model, X_test, y_test):
    y_test_pred_log = model.predict(X_test)

    y_test_pred_real = np.exp(y_test_pred_log)
    y_test_real = np.exp(y_test)

    mse_real = mean_squared_error(y_test_real, y_test_pred_real)
    r2_real = r2_score(y_test_real, y_test_pred_real)
    mae_real = mean_absolute_error(y_test_real, y_test_pred_real)
    medae_real = np.median(np.abs(y_test_real - y_test_pred_real))

    # MAPE — Mean Absolute Percentage Error
    mape = 100 * np.mean(np.abs((y_test_real - y_test_pred_real) / y_test_real))

    # MdAPE — Median Absolute Percentage Error
    mdape = 100 * np.median(np.abs((y_test_real - y_test_pred_real) / y_test_real))

    return mse_real, r2_real, mae_real, medae_real, mape, mdape

In [13]:
def crear_tabla_resultados(nombres, mse_list, r2_list, mae_list, medae_list, mape_list, mdape_list):
    resultados = []

    for nombre, mse, r2, mae, medae, mape, mdape in zip(nombres, mse_list, r2_list, mae_list, medae_list, mape, mdape_list):
        resultados.append({
            "Modelo": nombre,
            "MAE": mae,
            "MEDAE": medae,
            "MSE": mse,
            "R2": r2,
            "MAPE": mape,
            "MDAPE": mdape
        })

    df_resultados = pd.DataFrame(resultados)
    pd.options.display.float_format = '{:,.2f}'.format
    return df_resultados


## Target y Features

In [14]:
features_simple = ["Baños", "Dormitorio", "Superficie", "Superficie Construida", "Superficie Terreno", "Garage"]

features = ["Baños", "Dormitorio", "Superficie", "Superficie Construida", "Superficie Terreno", "Garage", "precio_por_m2_construido", "precio_por_m2_terreno", "precio_por_m2"]
target = "precio"

## Train Test Splits

In [94]:
X_train_all_municipios_simple, X_test_all_municipios_simple, y_train_all_municipios_simple, y_test_all_municipios_simple, encoder_all_municipios_simple, test_indices_all_municipios_simple = split_data_log_one_hot(
    data,
    target=target,
    variables_numericas=features_simple,
    variables_categoricas=[]
)

In [95]:
data_ch = data[data["municipio"] == "CH"].copy()

X_train_ch_simple, X_test_ch_simple, y_train_ch_simple, y_test_ch_simple, encoder_ch_simple, test_indices_ch_simple = split_data_log_one_hot(
    data_ch,
    target=target,
    variables_numericas=features_simple,
    variables_categoricas=[]
)

In [96]:
data_e = data[data["municipio"] == "E"].copy()

X_train_e_simple, X_test_e_simple, y_train_e_simple, y_test_e_simple, encoder_e_simple, test_indices_e_simple = split_data_log_one_hot(
    data_e,
    target=target,
    variables_numericas=features_simple,
    variables_categoricas=[]
)

In [97]:
X_train_all_municipios_completo, X_test_all_municipios_completo, y_train_all_municipios_completo, y_test_all_municipios_completo, encoder_all_municipios_completo, test_indices_all_municipios_completo = split_data_log_one_hot(
    data,
    target=target,
    variables_numericas=features,
    variables_categoricas=[]
)

In [98]:
data_ch = data[data["municipio"] == "CH"].copy()

X_train_ch_completo, X_test_ch_completo, y_train_ch_completo, y_test_ch_completo, encoder_ch_completo, test_indices_ch_completo = split_data_log_one_hot(
    data_ch,
    target=target,
    variables_numericas=features,
    variables_categoricas=[]
)

In [99]:
data_e = data[data["municipio"] == "E"].copy()

X_train_e_completo, X_test_e_completo, y_train_e_completo, y_test_e_completo, encoder_e_completo, test_indices_e_completo = split_data_log_one_hot(
    data_e,
    target=target,
    variables_numericas=features,
    variables_categoricas=[]
)

## Modelos

Entrenaremos modelos con features simples y features completas en todos los casos.

* Linear regression all_municipios
* RF municipio_ch
* RF municipio_e
* RF all_municipios
* RF Gridsearch municipio_ch
* RF Gridsearch municipio_e
* RF Gridsearch all_municipios

Nombres de modelos simples y completos:

* `model_lr_simple`, `model_lr`
* `rf_municipio_ch_simple`, `rf_municipio_ch`
* `rf_municipio_e_simple`, `rf_municipio_e`
* `rf_all_municipios_simple`, `rf_all_municipios`
* `rf_gridsearch_municipio_ch_simple`, `rf_gridsearch_municipio_ch`
* `rf_gridsearch_municipio_e_simple`, `rf_gridsearch_municipio_e`
* `rf_gridsearch_all_municipios_simple`, `rf_gridsearch_all_municipios`

### `model_lr_simple`

Todos los municipios, features simples

In [100]:
model_lr_simple = linear_regression_model(X_train_all_municipios_simple, y_train_all_municipios_simple)

print("Modelo entrenado:", model_lr_simple)

Modelo entrenado: LinearRegression()


In [101]:
mse_lr_simple, r2_lr_simple, mae_lr_simple, medae_lr_simple, mape_lr_simple, mdape_lr_simple  = evaluate_model(model_lr_simple, X_test_all_municipios_simple, y_test_all_municipios_simple)

print(f"Mean Squared Error (Real): {mse_lr_simple}")
print(f"R² Score (Real): {r2_lr_simple}")
print(f"Mean Absolute Error (Real): {mae_lr_simple}")
print(f"Median Absolute Error (Real): {medae_lr_simple}")
print(f"MAPE: {mape_lr_simple}")
print(f"Median Absolute Percentage Error (%): {mdape_lr_simple}")

Mean Squared Error (Real): 59582776668.79088
R² Score (Real): 0.5143294273566514
Mean Absolute Error (Real): 142291.40923812622
Median Absolute Error (Real): 81770.33876812772
MAPE: 43.48190743056862
Median Absolute Percentage Error (%): 28.508359762851903


### `model_lr`

In [102]:
model_lr = linear_regression_model(X_train_all_municipios_completo, y_train_all_municipios_completo)

print("Modelo entrenado:", model_lr)

Modelo entrenado: LinearRegression()


In [103]:
mse_lr_completo, r2_lr_completo, mae_lr_completo, medae_lr_completo, mape_lr_completo, mdape_lr_completo = evaluate_model(model_lr, X_test_all_municipios_completo, y_test_all_municipios_completo)

print(f"Mean Squared Error (Real): {mse_lr_completo}")
print(f"R² Score (Real): {r2_lr_completo}")
print(f"Mean Absolute Error (Real): {mae_lr_completo}")
print(f"Median Absolute Error (Real): {medae_lr_completo}")
print(f"MAPE: {mape_lr_completo}")
print(f"Median Absolute Percentage Error (%): {mdape_lr_completo}")

Mean Squared Error (Real): 62850851577.10847
R² Score (Real): 0.48769072568977645
Mean Absolute Error (Real): 122209.47705755077
Median Absolute Error (Real): 61997.05944371056
MAPE: 36.38061633599417
Median Absolute Percentage Error (%): 22.330725821491413


### `rf_municipio_ch_simple`

In [104]:
rf_municipio_ch_simple = train_random_forest(X_train_ch_simple, y_train_ch_simple)

print("Modelo entrenado:", rf_municipio_ch_simple)

Modelo entrenado: RandomForestRegressor(n_jobs=-1, random_state=42)


In [105]:
mse_rf_municipio_ch_simple, r2_rf_municipio_ch_simple, mae_rf_municipio_ch_simple, medae_rf_municipio_ch_simple, mape_rf_municipio_ch_simple, mdape_rf_municipio_ch_simple = evaluate_model(rf_municipio_ch_simple, X_test_ch_simple, y_test_ch_simple)

print(f"Mean Squared Error (Real): {mse_rf_municipio_ch_simple}")
print(f"R² Score (Real): {r2_rf_municipio_ch_simple}")
print(f"Mean Absolute Error (Real): {mae_rf_municipio_ch_simple}")
print(f"Median Absolute Error (Real): {medae_rf_municipio_ch_simple}")
print(f"MAPE: {mape_rf_municipio_ch_simple}")
print(f"Median Absolute Percentage Error (%): {mdape_rf_municipio_ch_simple}")

Mean Squared Error (Real): 9909051764.225548
R² Score (Real): 0.700828468511054
Mean Absolute Error (Real): 70627.1797401768
Median Absolute Error (Real): 47425.09686277274
MAPE: 19.543719850271724
Median Absolute Percentage Error (%): 13.519706508188689


### `rf_municipio_ch`

In [106]:
rf_municipio_ch = train_random_forest(X_train_ch_completo, y_train_ch_completo)

print("Modelo entrenado:", rf_municipio_ch)

Modelo entrenado: RandomForestRegressor(n_jobs=-1, random_state=42)


In [107]:
mse_rf_municipio_ch_completo, r2_rf_municipio_ch_completo, mae_rf_municipio_ch_completo, medae_rf_municipio_ch_completo, mape_rf_municipio_ch_completo, mdape_rf_municipio_ch_completo = evaluate_model(rf_municipio_ch, X_test_ch_completo, y_test_ch_completo)

print(f"Mean Squared Error (Real): {mse_rf_municipio_ch_completo}")
print(f"R² Score (Real): {r2_rf_municipio_ch_completo}")
print(f"Mean Absolute Error (Real): {mae_rf_municipio_ch_completo}")
print(f"Median Absolute Error (Real): {medae_rf_municipio_ch_completo}")
print(f"MAPE: {mape_rf_municipio_ch_completo}")
print(f"Median Absolute Percentage Error (%): {mdape_rf_municipio_ch_completo}")

Mean Squared Error (Real): 3478245182.4347353
R² Score (Real): 0.8949857198364957
Mean Absolute Error (Real): 34200.732218856945
Median Absolute Error (Real): 15205.900123070678
MAPE: 10.148082494443598
Median Absolute Percentage Error (%): 5.591072383030621


### `rf_municipio_e_simple`

municipio e y features simples

In [108]:
rf_municipio_e_simple = train_random_forest(X_train_e_simple, y_train_e_simple)

print("Modelo entrenado:", rf_municipio_e_simple)

Modelo entrenado: RandomForestRegressor(n_jobs=-1, random_state=42)


In [109]:
mse_rf_municipio_e_simple, r2_rf_municipio_e_simple, mae_rf_municipio_e_simple, medae_rf_municipio_e_simple, mape_rf_municipio_e_simple, mdape_rf_municipio_e_simple = evaluate_model(rf_municipio_e_simple, X_test_e_simple, y_test_e_simple)

print(f"Mean Squared Error (Real): {mse_rf_municipio_e_simple}")
print(f"R² Score (Real): {r2_rf_municipio_e_simple}")
print(f"Mean Absolute Error (Real): {mae_rf_municipio_e_simple}")
print(f"Median Absolute Error (Real): {medae_rf_municipio_e_simple}")
print(f"MAPE: {mape_rf_municipio_e_simple}")
print(f"Median Absolute Percentage Error (%): {mdape_rf_municipio_e_simple}")

Mean Squared Error (Real): 60643780573.23998
R² Score (Real): 0.5120008830220828
Mean Absolute Error (Real): 142054.6011102909
Median Absolute Error (Real): 82204.56496170483
MAPE: 38.61626301574597
Median Absolute Percentage Error (%): 16.621607280846835


### `rf_municipio_e`

In [110]:
rf_municipio_e = train_random_forest(X_train_e_completo, y_train_e_completo)

print("Modelo entrenado:", rf_municipio_e)

Modelo entrenado: RandomForestRegressor(n_jobs=-1, random_state=42)


In [111]:
mse_rf_municipio_e_completo, r2_rf_municipio_e_completo, mae_rf_municipio_e_completo, medae_rf_municipio_e_completo, mape_rf_municipio_e_completo, mdape_rf_municipio_e_completo = evaluate_model(rf_municipio_e, X_test_e_completo, y_test_e_completo)

print(f"Mean Squared Error (Real): {mse_rf_municipio_e_completo}")
print(f"R² Score (Real): {r2_rf_municipio_e_completo}")
print(f"Mean Absolute Error (Real): {mae_rf_municipio_e_completo}")
print(f"Median Absolute Error (Real): {medae_rf_municipio_e_completo}")
print(f"MAPE: {mape_rf_municipio_e_completo}")
print(f"Median Absolute Percentage Error (%): {mdape_rf_municipio_e_completo}")

Mean Squared Error (Real): 8377518411.217228
R² Score (Real): 0.9325863007138403
Mean Absolute Error (Real): 46633.75745563258
Median Absolute Error (Real): 21420.895272884547
MAPE: 14.085655567081693
Median Absolute Percentage Error (%): 5.188566399039236


### `rf_all_municipios_simple`

In [112]:
rf_all_municipios_simple = train_random_forest(X_train_all_municipios_simple, y_train_all_municipios_simple)

print("Modelo entrenado:", rf_all_municipios_simple)

Modelo entrenado: RandomForestRegressor(n_jobs=-1, random_state=42)


In [113]:
mse_rf_all_municipios_simple, r2_rf_all_municipios_simple, mae_rf_all_municipios_simple, medae_rf_all_municipios_simple, mape_rf_all_municipios_simple, mdape_rf_all_municipios_simple = evaluate_model(rf_all_municipios_simple, X_test_all_municipios_simple, y_test_all_municipios_simple)

print(f"Mean Squared Error (Real): {mse_rf_all_municipios_simple}")
print(f"R² Score (Real): {r2_rf_all_municipios_simple}")
print(f"Mean Absolute Error (Real): {mae_rf_all_municipios_simple}")
print(f"Median Absolute Error (Real): {medae_rf_all_municipios_simple}")
print(f"MAPE: {mape_rf_all_municipios_simple}")
print(f"Median Absolute Percentage Error (%): {mdape_rf_all_municipios_simple}")

Mean Squared Error (Real): 62201298533.44668
R² Score (Real): 0.49298535639205276
Mean Absolute Error (Real): 139621.40778687573
Median Absolute Error (Real): 74667.19691982535
MAPE: 42.20126069512958
Median Absolute Percentage Error (%): 26.38094963264618


`rf_all_municipios`

In [114]:
rf_all_municipios = train_random_forest(X_train_all_municipios_completo, y_train_all_municipios_completo)

print("Modelo entrenado:", rf_all_municipios)

Modelo entrenado: RandomForestRegressor(n_jobs=-1, random_state=42)


In [115]:
mse_rf_all_municipios_completo, r2_rf_all_municipios_completo, mae_rf_all_municipios_completo, medae_rf_all_municipios_completo, mape_rf_all_municipios_completo, mdape_rf_all_municipios_completo = evaluate_model(rf_all_municipios, X_test_all_municipios_completo, y_test_all_municipios_completo)

print(f"Mean Squared Error (Real): {mse_rf_all_municipios_completo}")
print(f"R² Score (Real): {r2_rf_all_municipios_completo}")
print(f"Mean Absolute Error (Real): {mae_rf_all_municipios_completo}")
print(f"Median Absolute Error (Real): {medae_rf_all_municipios_completo}")
print(f"MAPE: {mape_rf_all_municipios_completo}")
print(f"Median Absolute Percentage Error (%): {mdape_rf_all_municipios_completo}")

Mean Squared Error (Real): 12896809431.429754
R² Score (Real): 0.894875647426558
Mean Absolute Error (Real): 36929.371894682045
Median Absolute Error (Real): 12048.906506770873
MAPE: 9.152933637401926
Median Absolute Percentage Error (%): 4.603897535786204


### `rf_gridsearch_municipio_ch_simple`

In [37]:
rf_gridsearch_municipio_ch_simple = random_forest_grid_search(X_train_ch_simple, y_train_ch_simple)

print("Modelo entrenado:", rf_gridsearch_municipio_ch_simple)

Fitting 5 folds for each of 243 candidates, totalling 1215 fits


KeyboardInterrupt: 

In [ ]:
mse_rf_gridsearch_municipio_ch_simple, r2_rf_gridsearch_municipio_ch_simple, mae_rf_gridsearch_municipio_ch_simple, medae_rf_gridsearch_municipio_ch_simple, mape_rf_gridsearch_municipio_ch_simple, mdape_rf_gridsearch_municipio_ch_simple = evaluate_model(rf_gridsearch_municipio_ch_simple, X_train_ch_simple, y_train_ch_simple)

print(f"Mean Squared Error (Real): {mse_rf_gridsearch_municipio_ch_simple}")
print(f"R² Score (Real): {r2_rf_gridsearch_municipio_ch_simple}")
print(f"Mean Absolute Error (Real): {mae_rf_gridsearch_municipio_ch_simple}")
print(f"Median Absolute Error (Real): {medae_rf_gridsearch_municipio_ch_simple}")
print(f"MAPE: {mape_rf_gridsearch_municipio_ch_simple}")
print(f"Median Absolute Percentage Error (%): {mdape_rf_gridsearch_municipio_ch_simple}")

### `rf_gridsearch_municipio_ch`

In [ ]:
rf_gridsearch_municipio_ch = random_forest_grid_search(X_train_ch_completo, y_train_ch_completo)

print("Modelo entrenado:", rf_gridsearch_municipio_ch)

Fitting 5 folds for each of 243 candidates, totalling 1215 fits
Mejores parámetros: {'max_depth': None, 'max_features': 0.8, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 600}
Mejor score (CV MAE log): -0.13510193551057864
Modelo entrenado: RandomForestRegressor(max_features=0.8, n_estimators=600, n_jobs=-1,
                      random_state=42)


In [ ]:
mse_rf_gridsearch_municipio_ch_completo, r2_rf_gridsearch_municipio_ch_completo, mae_rf_gridsearch_municipio_ch_completo, medae_rf_gridsearch_municipio_ch_completo, mape_rf_gridsearch_municipio_ch_completo, mdape_rf_gridsearch_municipio_ch_completo = evaluate_model(rf_gridsearch_municipio_ch, X_train_ch_completo, y_train_ch_completo)

print(f"Mean Squared Error (Real): {mse_rf_gridsearch_municipio_ch_completo}")
print(f"R² Score (Real): {r2_rf_gridsearch_municipio_ch_completo}")
print(f"Mean Absolute Error (Real): {mae_rf_gridsearch_municipio_ch_completo}")
print(f"Median Absolute Error (Real): {medae_rf_gridsearch_municipio_ch_completo}")
print(f"MAPE: {mape_rf_gridsearch_municipio_ch_completo}")
print(f"Median Absolute Percentage Error (%): {mdape_rf_gridsearch_municipio_ch_completo}")

Mean Squared Error (Real): 2497618818.1668377
R² Score (Real): 0.9660217656177834
Mean Absolute Error (Real): 22194.821650444494
Median Absolute Error (Real): 10460.52170110424
MAPE: 4.773753953015525
Median Absolute Percentage Error (%): 3.1264731904619008


### `rf_gridsearch_municipio_e_simple`

In [ ]:
rf_gridsearch_municipio_e_simple = random_forest_grid_search(X_train_e_simple, y_train_e_simple)

print("Modelo entrenado:", rf_gridsearch_municipio_e_simple)

Fitting 5 folds for each of 243 candidates, totalling 1215 fits
Mejores parámetros: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Mejor score (CV MAE log): -0.2801139297676653
Modelo entrenado: RandomForestRegressor(max_depth=10, max_features='sqrt', n_estimators=200,
                      n_jobs=-1, random_state=42)


In [ ]:
mse_rf_gridsearch_municipio_e_simple, r2_rf_gridsearch_municipio_e_simple, mae_rf_gridsearch_municipio_e_simple, medae_rf_gridsearch_municipio_e_simple, mape_rf_gridsearch_municipio_e_simple, mdape_rf_gridsearch_municipio_e_simple = evaluate_model(rf_gridsearch_municipio_e_simple, X_train_e_simple, y_train_e_simple)

print(f"Mean Squared Error (Real): {mse_rf_gridsearch_municipio_e_simple}")
print(f"R² Score (Real): {r2_rf_gridsearch_municipio_e_simple}")
print(f"Mean Absolute Error (Real): {mae_rf_gridsearch_municipio_e_simple}")
print(f"Median Absolute Error (Real): {medae_rf_gridsearch_municipio_e_simple}")
print(f"MAPE: {mape_rf_gridsearch_municipio_e_simple}")
print(f"Median Absolute Percentage Error (%): {mdape_rf_gridsearch_municipio_e_simple}")

Mean Squared Error (Real): 16661228987.960726
R² Score (Real): 0.884042258366629
Mean Absolute Error (Real): 80507.09043697249
Median Absolute Error (Real): 44958.98967089428
MAPE: 12.343585763302702
Median Absolute Percentage Error (%): 8.915469758940892


### `rf_gridsearch_municipio_e`

In [ ]:
rf_gridsearch_municipio_e = random_forest_grid_search(X_train_e_completo, y_train_e_completo)

print("Modelo entrenado:", rf_gridsearch_municipio_e)

Fitting 5 folds for each of 243 candidates, totalling 1215 fits
Mejores parámetros: {'max_depth': None, 'max_features': 0.8, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 400}
Mejor score (CV MAE log): -0.11658899678731979
Modelo entrenado: RandomForestRegressor(max_features=0.8, n_estimators=400, n_jobs=-1,
                      random_state=42)


In [ ]:
mse_rf_gridsearch_municipio_e_completo, r2_rf_gridsearch_municipio_e_completo, mae_rf_gridsearch_municipio_e_completo, medae_rf_gridsearch_municipio_e_completo, mape_rf_gridsearch_municipio_e_completo, mdape_rf_gridsearch_municipio_e_completo = evaluate_model(rf_gridsearch_municipio_e, X_train_e_completo, y_train_e_completo)

print(f"Mean Squared Error (Real): {mse_rf_gridsearch_municipio_e_completo}")
print(f"R² Score (Real): {r2_rf_gridsearch_municipio_e_completo}")
print(f"Mean Absolute Error (Real): {mae_rf_gridsearch_municipio_e_completo}")
print(f"Median Absolute Error (Real): {medae_rf_gridsearch_municipio_e_completo}")
print(f"MAPE: {mape_rf_gridsearch_municipio_e_completo}")
print(f"Median Absolute Percentage Error (%): {mdape_rf_gridsearch_municipio_e_completo}")

Mean Squared Error (Real): 5234476696.774931
R² Score (Real): 0.9635694283519465
Mean Absolute Error (Real): 28270.245211461188
Median Absolute Error (Real): 8794.236630378524
MAPE: 3.9480670766645622
Median Absolute Percentage Error (%): 1.8704571624022714


### `rf_gridsearch_all_municipios_simple`

In [ ]:
rf_gridsearch_all_municipios_simple = random_forest_grid_search(X_train_all_municipios_simple, y_train_all_municipios_simple)

print("Modelo entrenado:", rf_gridsearch_all_municipios_simple)

In [ ]:
mse_rf_gridsearch_all_municipios_simple, r2_rf_gridsearch_all_municipios_simple, mae_rf_gridsearch_all_municipios_simple, medae_rf_gridsearch_all_municipios_simple, mape_rf_gridsearch_all_municipios_simple, mdape_rf_gridsearch_all_municipios_simple = evaluate_model(rf_gridsearch_all_municipios_simple, X_train_all_municipios_simple, y_train_all_municipios_simple)

print(f"Mean Squared Error (Real): {mse_rf_gridsearch_all_municipios_simple}")
print(f"R² Score (Real): {r2_rf_gridsearch_all_municipios_simple}")
print(f"Mean Absolute Error (Real): {mae_rf_gridsearch_all_municipios_simple}")
print(f"Median Absolute Error (Real): {medae_rf_gridsearch_all_municipios_simple}")
print(f"MAPE: {mape_rf_gridsearch_all_municipios_simple}")
print(f"Median Absolute Percentage Error (%): {mdape_rf_gridsearch_all_municipios_simple}")

### `rf_gridsearch_all_municipios`

In [ ]:
rf_gridsearch_all_municipios = random_forest_grid_search(X_train_all_municipios_completo, y_train_all_municipios_completo)

print("Modelo entrenado:", rf_gridsearch_all_municipios)

In [ ]:
mse_rf_gridsearch_all_municipios_completo, r2_rf_gridsearch_all_municipios_completo, mae_rf_gridsearch_all_municipios_completo, medae_rf_gridsearch_all_municipios_completo, mape_rf_gridsearch_all_municipios_completo, mdape_rf_gridsearch_all_municipios_completo = evaluate_model(rf_gridsearch_all_municipios, X_train_all_municipios_completo, y_train_all_municipios_completo)

print(f"Mean Squared Error (Real): {mse_rf_gridsearch_all_municipios_completo}")
print(f"R² Score (Real): {r2_rf_gridsearch_all_municipios_completo}")
print(f"Mean Absolute Error (Real): {mae_rf_gridsearch_all_municipios_completo}")
print(f"Median Absolute Error (Real): {medae_rf_gridsearch_all_municipios_completo}")
print(f"MAPE: {mape_rf_gridsearch_all_municipios_completo}")
print(f"Median Absolute Percentage Error (%): {mdape_rf_gridsearch_all_municipios_completo}")

## Model Performance

* `model_lr_categoricas`: refiere a Linear Regression con target `precio` y con

In [ ]:
all_models_simple = [model_lr_simple, rf_municipio_ch_simple, rf_municipio_e_simple, rf_all_municipios_simple, rf_gridsearch_municipio_ch_simple, rf_gridsearch_municipio_e_simple, rf_gridsearch_all_municipios_simple]
mse_models_simple = [mse_lr_simple, mse_rf_municipio_ch_simple, mse_rf_municipio_e_simple, mse_rf_all_municipios_simple, mse_rf_gridsearch_municipio_ch_simple, mse_rf_gridsearch_municipio_e_simple, mse_rf_gridsearch_all_municipios_simple]
r2_models_simple = [r2_lr_simple, r2_rf_municipio_ch_simple, r2_rf_municipio_e_simple, r2_rf_all_municipios_simple, r2_rf_gridsearch_municipio_ch_simple, r2_rf_gridsearch_municipio_e_simple, r2_rf_gridsearch_all_municipios_simple]
mae_models_simple = [mae_lr_simple, mae_rf_municipio_ch_simple, mae_rf_municipio_e_simple, mae_rf_all_municipios_simple, mae_rf_gridsearch_municipio_ch_simple, mae_rf_gridsearch_municipio_e_simple, mae_rf_gridsearch_all_municipios_simple]
medae_models_simple = [medae_lr_simple, medae_rf_municipio_ch_simple, medae_rf_municipio_e_simple, medae_rf_all_municipios_simple, medae_rf_gridsearch_municipio_ch_simple, medae_rf_gridsearch_municipio_e_simple, medae_rf_gridsearch_all_municipios_simple]
mape_models_simple = [mape_lr_simple, mape_rf_municipio_ch_simple, mape_rf_municipio_e_simple, mape_rf_all_municipios_simple, mape_rf_gridsearch_municipio_ch_simple, mape_rf_gridsearch_municipio_e_simple, mape_rf_gridsearch_all_municipios_simple]
mdape_models_simple = [mdape_lr_simple, mdape_rf_municipio_ch_simple, mdape_rf_municipio_e_simple, mdape_rf_all_municipios_simple, mdape_rf_gridsearch_municipio_ch_simple, mdape_rf_gridsearch_municipio_e_simple, mdape_rf_gridsearch_all_municipios_simple]

In [ ]:
all_models = [model_lr, rf_municipio_ch, rf_municipio_e, rf_all_municipios, rf_gridsearch_municipio_ch, rf_gridsearch_municipio_e, rf_gridsearch_all_municipios]
mse_models = [mse_lr_completo, mse_rf_municipio_ch_completo, mse_rf_municipio_e_completo, mse_rf_all_municipios_completo, mse_rf_gridsearch_municipio_ch_completo, mse_rf_gridsearch_municipio_e_completo, mse_rf_gridsearch_all_municipios_completo]
r2_models = [r2_lr_completo, r2_rf_municipio_ch_completo, r2_rf_municipio_e_completo, r2_rf_all_municipios_completo, r2_rf_gridsearch_municipio_ch_completo, r2_rf_gridsearch_municipio_e_completo, r2_rf_gridsearch_all_municipios_completo]
mae_models = [mae_lr_completo, mae_rf_municipio_ch_completo, mae_rf_municipio_e_completo, mae_rf_all_municipios_completo, mae_rf_gridsearch_municipio_ch_completo, mae_rf_gridsearch_municipio_e_completo, mae_rf_gridsearch_all_municipios_completo]
medae_models = [medae_lr_completo, medae_rf_municipio_ch_completo, medae_rf_municipio_e_completo, medae_rf_all_municipios_completo, medae_rf_gridsearch_municipio_ch_completo, medae_rf_gridsearch_municipio_e_completo, medae_rf_gridsearch_all_municipios_completo]
mape_models = [mape_lr_completo, mape_rf_municipio_ch_completo, mape_rf_municipio_e_completo, mape_rf_all_municipios_completo, mape_rf_gridsearch_municipio_ch_completo, mape_rf_gridsearch_municipio_e_completo, mape_rf_gridsearch_all_municipios_completo]
mdape_models = [mdape_lr_completo, mdape_rf_municipio_ch_completo, mdape_rf_municipio_e_completo, mdape_rf_all_municipios_completo, mdape_rf_gridsearch_municipio_ch_completo, mdape_rf_gridsearch_municipio_e_completo, mdape_rf_gridsearch_all_municipios_completo]

In [ ]:
# Nombres de los modelos (los mismos para simple y normal)
model_names = [
    "LR",
    "RF municipio_ch",
    "RF municipio_e",
    "RF all municipios",
    "RF GridSearch municipio_ch",
    "RF GridSearch municipio_e",
    "RF Gridsearch all_municipios"
]

# DataFrame para los modelos simples
tabla_simple = crear_tabla_resultados(model_names, mse_models_simple, r2_models_simple, mae_models_simple, medae_models_simple, mape_models_simple, mdape_models_simple)

# DataFrame para los modelos completos
tabla = crear_tabla_resultados(model_names, mse_models, r2_models, mae_models, medae_models, mape_models, mdape_models)

print("Resultados modelos simples:")
tabla_simple.head(10)

In [ ]:
print("Resultados modelos completos:")
tabla.head(10)

## Analisis de resultados

In [120]:
extra_cols = ['url']

# Asegurate de que los índices coincidan
X_test = X_test_all_municipios_completo.reset_index(drop=True)
y_test = y_test_all_municipios_completo.reset_index(drop=True)

# Ejecutar la función
results = evaluation_table(
    rf_all_municipios,
    X_test,
    y_test,
    data.iloc[test_indices_all_municipios_completo],
    extra_cols
)
# Ver los primeros resultados
print("RF All Municipios Completo: \n")
results.head(20)

RF All Municipios Completo: 



,Precio Real,Precio Predicho,Error Absoluto,Error %,url
0,55000.0,126252.438875,7.125244e+04,129.549889,https://www.casasymas.com.uy/propiedad/120273-...
1,2000000.0,688249.676478,1.311750e+06,65.587516,https://www.casasymas.com.uy/propiedad/1322-ca...
2,305000.0,146116.587839,1.588834e+05,52.092922,https://www.casasymas.com.uy/propiedad/116984-...
3,170000.0,258149.601599,8.814960e+04,51.852707,https://www.casasymas.com.uy/propiedad/207263-...
4,120000.0,174765.090940,5.476509e+04,45.637576,https://www.casasymas.com.uy/propiedad/199197-...
5,157000.0,218146.261411,6.114626e+04,38.946663,https://www.casasymas.com.uy/propiedad/203188-...
6,420000.0,261634.969294,1.583650e+05,37.705960,https://www.casasymas.com.uy/propiedad/38319-c...
7,128000.0,173473.618039,4.547362e+04,35.526264,https://www.casasymas.com.uy/propiedad/151589-...
8,850000.0,580719.392520,2.692806e+05,31.680071,https://www.casasymas.com.uy/propiedad/65438-c...
9,175000.0,120110.113893,5.488989e+04,31.365649,https://www.casasymas.com.uy/propiedad/64480-c...


In [47]:
umbral_10 = 10
umbral_20 = 20
umbral_5 = 5

# Contar
count_below_5 = (results["Error %"] <= umbral_5).sum()
count_below_10 = (results["Error %"] <= umbral_10).sum()
count_above_5 = (results["Error %"] > umbral_5).sum()
count_above_10 = (results["Error %"] > umbral_10).sum()

# Mostrar resultados
print(f"Predicciones con error ≤ {umbral_5}%: {count_below_5}")
print(f"Predicciones con error ≤ {umbral_10}%: {count_below_10}")
print(f"Predicciones con error > {umbral_5}%: {count_above_5}")
print(f"Predicciones con error > {umbral_10}%: {count_above_10}")

# Si querés también el porcentaje respecto al total:
total = len(results)
print("\nPorcentajes:")
print(f"≤ {umbral_5}%: {count_below_5 / total * 100:.2f}%")
print(f"≤ {umbral_10}%: {count_below_10 / total * 100:.2f}%")
print(f"> {umbral_5}%: {count_above_5 / total * 100:.2f}%")
print(f"> {umbral_10}%: {count_above_10 / total * 100:.2f}%")

Predicciones con error ≤ 5%: 98
Predicciones con error ≤ 10%: 141
Predicciones con error > 5%: 91
Predicciones con error > 10%: 48

Porcentajes:
≤ 5%: 51.85%
≤ 10%: 74.60%
> 5%: 48.15%
> 10%: 25.40%


In [68]:
y_test_all_municipios_completo.head(10)

,precio
96,14.403297
265,12.254863
811,12.524526
23,13.304685
30,11.759786
280,12.847927
569,12.676076
259,13.604790
331,12.367341
323,12.793859
